# Train model

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import psycopg2

In [16]:
conn = psycopg2.connect(
    host="movie-warehouse.postgres.database.azure.com", dbname="warehouse", user="user", password="...", sslmode="require"
)

In [17]:
sql = """
WITH tmp as 
(
    SELECT castlistfk as movie_id, STRING_AGG(name, ', ') as cast FROM castlist GROUP BY movie_id
),

tmp2 as (
    SELECT crewlistfk as movie_id, STRING_AGG(name, ', ') as crew FROM crewlist GROUP BY movie_id
), 


tmp3 as (
    SELECT tmp.*, tmp2.crew FROM tmp JOIN tmp2 ON tmp.movie_id = tmp2.movie_id 
)

SELECT movies.*, tmp3.cast, tmp3.crew FROM movies JOIN tmp3 USING(movie_id) WHERE numofvotes > 200
"""
movies = pd.read_sql_query(sql, con=conn)
movies

C:\Users\anczo\AppData\Local\Temp\ipykernel_13048\2900340554.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies = pd.read_sql_query(sql, con=conn)


,movie_id,title,year,isadult,runtime,genre1,genre2,genre3,crewnumber,castnumber,avgrating,numofvotes,cast,crew
0,184,Safety Last!,1923,0,74,Action,Comedy,Thriller,7,3,3.918,457,"Harold Lloyd, Bill Strother, Noah Young","Fred C. Newmeyer, Sam Taylor, Hal Roach, Tim W..."
1,292,The Lodger: A Story of the London Fog,1927,0,92,Crime,Drama,Mystery,3,2,3.538,274,"Ivor Novello, Arthur Chesney","Alfred Hitchcock, Marie Belloc Lowndes, Eliot ..."
2,240,Go West,1925,0,69,Comedy,Western,\N,3,3,3.633,260,"Buster Keaton, Howard Truesdale, Ray Thompson","Buster Keaton, Lex Neal, Raymond Cannon"
3,328,The General,1926,0,67,Action,Adventure,Comedy,6,3,4.101,2695,"Buster Keaton, Glen Cavender, Jim Farley","Buster Keaton, Clyde Bruckman, Al Boasberg, Ch..."
4,54,Intolerance: Love's Struggle Throughout the Ages,1916,0,163,Drama,History,\N,7,2,3.660,241,"Robert Harron, F.A. Turner","D.W. Griffith, Hettie Grey Baker, Tod Browning..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533,24107,Ultraviolet,2006,0,88,Action,Sci-Fi,\N,1,3,2.442,1776,"Cameron Bright, Nick Chinlund, Sebastien Andrieu",Kurt Wimmer
7534,12586,Stop Making Sense,1984,0,88,Documentary,Music,\N,2,4,4.097,2452,"David Byrne, Bernie Worrell, Alex Weir, Steven...","Jonathan Demme, Talking Heads"
7535,30935,Seeking a Friend for the End of the World,2012,0,101,Adventure,Comedy,Drama,1,2,3.430,1354,"Steve Carell, Patton Oswalt",Lorene Scafaria
7536,28494,Zombie Strippers!,2008,0,94,Comedy,Horror,Sci-Fi,1,1,2.128,238,Robert Englund,Jay Lee


In [18]:
sql = """
WITH tmp as 
(
    SELECT userId FROM ratinglist GROUP BY userId HAVING COUNT(*) > 25
)

SELECT list.userid, list.rating, list.ratinglistfk as movie_id FROM tmp LEFT JOIN ratinglist as list USING(userId) WHERE ratinglistfk IN (SELECT movie_id FROM movies WHERE numofvotes > 150)
"""

ratings = pd.read_sql_query(sql, con=conn)
ratings

C:\Users\anczo\AppData\Local\Temp\ipykernel_13048\1470539357.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(sql, con=conn)


,userid,rating,movie_id
0,181291,3.0,30129
1,181343,4.0,30129
2,181712,3.0,30129
3,182408,4.0,30129
4,184947,3.0,30129
...,...,...,...
25112178,175237,3.0,30129
25112179,175313,2.0,30129
25112180,175927,3.0,30129
25112181,179605,2.0,30129


In [19]:
final_dataset = ratings.pivot(index='movie_id',columns='userid',values='rating')
final_dataset

userid,4,5,6,8,10,14,15,16,18,19,...,283210,283213,283214,283215,283218,283219,283221,283222,283224,283228
movie_id,,,,,,,,,,,,,,,,,,,,,
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
final_dataset.to_numpy()[final_dataset.isna()] = 0
final_dataset

userid,4,5,6,8,10,14,15,16,18,19,...,283210,283213,283214,283215,283218,283219,283221,283222,283224,283228
movie_id,,,,,,,,,,,,,,,,,,,,,
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
no_movies_voted = ratings.groupby('userid')['rating'].agg('count')
no_user_voted = ratings.groupby('movie_id')['rating'].agg('count')

In [22]:
final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 150].index,:]
final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
final_dataset

userid,4,5,10,14,15,18,19,26,31,34,...,283199,283204,283206,283208,283210,283215,283219,283222,283224,283228
movie_id,,,,,,,,,,,,,,,,,,,,,
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)

In [147]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=30)

In [48]:
def get_movie_recommendation(movies1):
    n_movies_to_reccomend = 10
    df_final = pd.DataFrame([], columns=['Title','Distance'])

    for movie_name in movies1:
        movie_list = movies[movies['title'] == movie_name]

        if len(movie_list):        
            movie_idx= movie_list.iloc[0]['movie_id']
            movie_idx = final_dataset[final_dataset['movie_id'] == movie_idx].index[0]
            distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
            rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
            recommend_frame = []
            for val in rec_movie_indices:
                movie_idx = final_dataset.iloc[val[0]]['movie_id']
                idx = movies[movies['movie_id'] == movie_idx].index
            
                recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
            df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
            df_final= df_final.append(df)
    
    
    df_final = df_final.groupby('Title').mean().reset_index()
    df_final = df_final[~df_final['Title'].isin(movies1)].sort_values(by=['Distance'], ascending=True)
    
    return df_final


In [49]:
get_movie_recommendation(["Harry Potter and the Half-Blood Prince"])['Title'].tolist()

C:\Users\anczo\AppData\Local\Temp\ipykernel_13048\269033938.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final= df_final.append(df)


['Harry Potter and the Deathly Hallows: Part 1',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Chamber of Secrets',
 "Harry Potter and the Sorcerer's Stone",
 'The Hunger Games',
 'Avatar',
 'The Avengers']

In [159]:
import pickle 
f = open('trainedModel.pkl', 'wb')
pickle.dump(knn, f)
f.close()

# Deploy model

In [8]:
from azureml.core import Workspace
import mlflow
ws = Workspace.from_config()

In [9]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [10]:
mlflow.set_experiment('testxxx')

2022/11/20 14:46:59 INFO mlflow.tracking.fluent: Experiment with name 'testxxx' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1668952043758, experiment_id='2e34bbea-4b0d-4b00-a8e7-72ecfddc13bc', last_update_time=None, lifecycle_stage='active', name='testxxx', tags={}>

In [15]:
mlflow.sklearn.autolog(max_tuning_runs=None)

In [46]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=40, n_jobs=-1)
knn.fit(csr_data)

2022/11/20 16:27:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fc05eef0-4f64-4ad7-8354-617239db3f91', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/11/20 16:27:19 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2022/11/20 16:27:19 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature
2022/11/20 16:27:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=40)

In [50]:
from azureml.core import Experiment, Workspace
experiment = Experiment(ws, 'testxxx')

In [51]:
run_id = 'fc05eef0-4f64-4ad7-8354-617239db3f91' #replace with own id
run = [r for r in experiment.get_runs() if r.id == run_id][0]

In [52]:
model = run.register_model(model_name = 'model_rec', model_path = 'model/model.pkl')

In [53]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [56]:
from azureml.core.model import Model
model = Model(ws, 'model_rec', version=2)

In [57]:
from azureml.core import Environment
env = Environment.from_conda_specification(name='recommendations', file_path="./azure/conda.yaml")
env.register(ws)

{
    "assetId": "azureml://locations/northeurope/workspaces/1e69dd1c-abc9-4891-a248-28b99473e69a/environments/recommendations/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "recommendations",